In [1]:
'''
科学计算final
韩秋原 
202111069028
'''
from PySide2.QtCore import QFile 
from PySide2.QtWidgets import QApplication, QMainWindow, QSpinBox, QPushButton, \
    QPlainTextEdit, QMessageBox, QLabel, QFileDialog, QHBoxLayout,QGraphicsScene
from PySide2.QtUiTools import QUiLoader
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LightSource
from matplotlib import cm
import numpy as np
import sys
from scipy import interpolate
import Logic
matplotlib.use("Qt5Agg")  # 声明使用QT5

In [29]:
class MyFigureCanvas(FigureCanvas): 
    def __init__(self, width=10, height=5,xlim=None, ylim=None, dpi=100,x_label='',y_label='',title='',colorbar=False):
        self.fig = Figure(figsize=(width, height),tight_layout=True)  
        FigureCanvas.__init__(self, self.fig)
        self.axes = self.fig.add_subplot(111)
        self.axes.spines['top'].set_visible(False)  # 去掉绘图时上面的横线
        self.axes.spines['right'].set_visible(False)  # 去掉绘图时右面的横线
        if colorbar:
            fig.colorbar(cm.ScalarMappable(),ax=self.axes)
        if xlim:
            self.axes.set_xlim(xlim)
        if ylim:
            self.axes.set_ylim(ylim)
        self.axes.set_xlabel(x_label)
        self.axes.set_ylabel(y_label)
        self.axes.set_title(title)
class MyFigureCanvas3D(FigureCanvas): 
    def __init__(self, width=10, height=5,xlim=None, ylim=None, dpi=100,x_label='',y_label='',title=''):
        fig = Figure(figsize=(width, height),tight_layout=True)  
        FigureCanvas.__init__(self, fig)
        self.axes = fig.add_subplot(111,projection='3d')
        self.axes.spines['top'].set_visible(False)  # 去掉绘图时上面的横线
        self.axes.spines['right'].set_visible(False)  # 去掉绘图时右面的横线
        if xlim:
            self.axes.set_xlim(xlim)
        if ylim:
            self.axes.set_ylim(ylim)
        self.axes.set_xlabel(x_label)
        self.axes.set_ylabel(y_label)
        self.axes.set_title(title)
#         print(type(self.axes))
        
        
class Stats:
    def __init__(self):
        self.ui = QUiLoader().load('ui/DEM.ui')
        self.ui.actionOpen.triggered.connect(self.HandleAddFile)
        self.ui.actionDEM_2.triggered.connect(lambda:self.HandlePlotHist('DEM'))
        self.ui.actionSlope_2.triggered.connect(lambda:self.HandlePlotHist('Slope'))
        self.ui.actionAspect_2.triggered.connect(lambda:self.HandlePlotHist('Aspect'))
        self.ui.actionDEM.triggered.connect(lambda:self.HandlePlot2D('DEM'))
        self.ui.actionSlope.triggered.connect(lambda:self.HandlePlot2D('Slope'))
        self.ui.actionAspect.triggered.connect(lambda:self.HandlePlot2D('Aspect'))
        self.ui.actionDEM_3.triggered.connect(lambda:self.HandlePlot3D('DEM'))
#         self.ui.button.clicked.connect(self.handleCalc)
        
    def HandleAddFile(self):
#         filename, fileType = QFileDialog.getOpenFileName(self.ui,"Open DEM")
#         print(fileType)
        filename='曹坪.asc'
        info,data=Logic.read_asc(filename)
        self.DEM_open= Logic.DEM(data,info)
    def HandlePlot2D(self,name):
        data_2d=self.DEM_open.plot_data_2d(name)
        data_min=np.min(data_2d)
        data_max=np.max(data_2d)
        title=name+' 2D graph'
        x_label='x axis'
        y_label='y axis'
        self.gv_visual_data_content = MyFigureCanvas(width=self.ui.graphicsView_mid.width() / 80,
                                                     height=self.ui.graphicsView_mid.height() / 80,
                                                     x_label=x_label,
                                                     y_label=y_label,
                                                     title=title)
        self.gv_visual_data_content.axes.matshow(data_2d)
        norm = matplotlib.colors.Normalize(vmin=data_min, vmax=data_max)
        self.gv_visual_data_content.fig.colorbar(cm.ScalarMappable(norm),ax=self.gv_visual_data_content.axes)
#         self.gv_visual_data_content.axes.colorbar()
        self.graphic_scene = QGraphicsScene()
        self.graphic_scene.addWidget(self.gv_visual_data_content)
        self.ui.graphicsView_mid.setScene(self.graphic_scene)  # 把QGraphicsScene放入QGraphicsView
        self.ui.graphicsView_mid.show()
    def HandlePlot3D(self,name):
        data_2d=self.DEM_open.plot_data_2d(name)
        data_min=self.DEM_open.export_min()
        data_max=self.DEM_open.export_max()
        data_2d[data_2d.mask]=data_min-500
        title=name+' 3D graph'
        x_label='x axis'
        y_label='y axis'
        self.gv_visual_data_content = MyFigureCanvas3D(width=self.ui.graphicsView_mid.width() / 80,
                                                     height=self.ui.graphicsView_mid.height() / 80,
                                                     x_label=x_label,
                                                     y_label=y_label,
                                                     title=title)
        print(data_2d.shape)
        x=data_2d.shape[1]
        y=data_2d.shape[0]
        X=np.linspace(0,x,x)
        Y=np.linspace(0,y,y)
        
        Z=data_2d
        f = interpolate.interp2d(X, Y, Z, kind='cubic')
        X=np.linspace(0,x,x)
        Y=np.linspace(0,y,y)
        Z=f(X,Y)
        X,Y=np.meshgrid(X,Y)
        print(X.shape,Y.shape,Z.shape,data_max-data_min)
#         ls = LightSource(270, 20) #设置你可视化数据的色带
#         rgb = ls.shade(Z, cmap=cm.gist_earth, vert_exag=0.1, blend_mode='soft')
        self.gv_visual_data_content.axes.plot_surface(X,Y,Z,rstride=3, cstride=3,
                                                      cmap=plt.get_cmap('rainbow'))
        self.graphic_scene = QGraphicsScene()
        self.graphic_scene.addWidget(self.gv_visual_data_content)
        self.ui.graphicsView_mid.setScene(self.graphic_scene)  # 把QGraphicsScene放入QGraphicsView
        self.ui.graphicsView_mid.show()
    def HandlePlotHist(self,name):
        x,y,bins,max_y,min_y=self.DEM_open.plot_hist(name)
        title='The histgram of '+name
        x_label='x axis'
        self.gv_visual_data_content = MyFigureCanvas(width=self.ui.graphicsView_R.width() / 80,
                                                     height=self.ui.graphicsView_R.height() / 80,
                                                     xlim=(max_y,min_y),
                                                     x_label=x_label,
                                                     y_label='number',
                                                     title=title)
        n,bins,patches=self.gv_visual_data_content.axes.hist(y,bins)
        self.gv_visual_data_content.axes.plot(x,n)
        self.graphic_scene = QGraphicsScene()
        self.graphic_scene.addWidget(self.gv_visual_data_content)
        self.ui.graphicsView_R.setScene(self.graphic_scene)  # 把QGraphicsScene放入QGraphicsView
        self.ui.graphicsView_R.show()
    def plot_DEM(self):
        pass
        
        
app = QApplication.instance()
if app is None: 
    app = QApplication(sys.argv)
stats = Stats()
stats.ui.show()
app.exec_()

(346, 395)
(346, 395) (346, 395) (346, 395) 380.0


0

In [15]:
A=QGraphicsScene()

In [16]:
A.addWidget?

Signature:
A.addWidget(
    self,
    widget:PySide2.QtWidgets.QWidget,
    wFlags:PySide2.QtCore.Qt.WindowFlags=Default(Qt.WindowFlags),
) -> PySide2.QtWidgets.QGraphicsProxyWidget
Docstring: addWidget(self, widget:PySide2.QtWidgets.QWidget, wFlags:PySide2.QtCore.Qt.WindowFlags=Default(Qt.WindowFlags)) -> PySide2.QtWidgets.QGraphicsProxyWidget
Type:      builtin_function_or_method


In [16]:
A=MyFigureCanvas()


Signature: A.axes.matshow(Z, **kwargs)
Docstring:
Plot the values of a 2D matrix or array as color-coded image.

The matrix will be shown the way it would be printed, with the first
row at the top.  Row and column numbering is zero-based.

Parameters
----------
Z : array-like(M, N)
    The matrix to be displayed.

Returns
-------
`~matplotlib.image.AxesImage`

Other Parameters
----------------
**kwargs : `~matplotlib.axes.Axes.imshow` arguments

See Also
--------
imshow : More general function to plot data on a 2D regular raster.

Notes
-----
This is just a convenience function wrapping `.imshow` to set useful
defaults for displaying a matrix. In particular:

- Set ``origin='upper'``.
- Set ``interpolation='nearest'``.
- Set ``aspect='equal'``.
- Ticks are placed to the left and above.
- Ticks are formatted to show integer indices.
File:      d:\anaconda3\envs\tensorflow\lib\site-packages\matplotlib\axes\_axes.py
Type:      method


In [18]:
import PySide6

In [10]:
fig=Figure()
fig.colorbar?

Signature: fig.colorbar(mappable, cax=None, ax=None, use_gridspec=True, **kw)
Docstring:
Create a colorbar for a ScalarMappable instance, *mappable*.

Documentation for the pyplot thin wrapper:


Add a colorbar to a plot.

Function signatures for the :mod:`~matplotlib.pyplot` interface; all
but the first are also method signatures for the `~.Figure.colorbar` method::

  colorbar(**kwargs)
  colorbar(mappable, **kwargs)
  colorbar(mappable, cax=cax, **kwargs)
  colorbar(mappable, ax=ax, **kwargs)

Parameters
----------
mappable
    The `matplotlib.cm.ScalarMappable` (i.e., `~matplotlib.image.AxesImage`,
    `~matplotlib.contour.ContourSet`, etc.) described by this colorbar.
    This argument is mandatory for the `.Figure.colorbar` method but optional
    for the `.pyplot.colorbar` function, which sets the default to the current
    image.

    Note that one can create a `.ScalarMappable` "on-the-fly" to generate
    colorbars not attached to a previously drawn artist, e.g. ::

        f

In [21]:
matplotlib.plot_surface?

Object `matplotlib.plot_surface` not found.


In [12]:
plt.get_cmap('rainbow')

In [22]:
cm.ScalarMappable?

Init signature: cm.ScalarMappable(norm=None, cmap=None)
Docstring:     
A mixin class to map scalar data to RGBA.

The ScalarMappable applies data normalization before returning RGBA colors
from the given colormap.
Init docstring:
Parameters
----------
norm : `matplotlib.colors.Normalize` (or subclass thereof)
    The normalizing object which scales data, typically into the
    interval ``[0, 1]``.
    If *None*, *norm* defaults to a *colors.Normalize* object which
    initializes its scaling based on the first data processed.
cmap : str or `~matplotlib.colors.Colormap`
    The colormap used to map normalized data values to RGBA colors.
File:           d:\anaconda3\envs\tensorflow\lib\site-packages\matplotlib\cm.py
Type:           type
Subclasses:     _ImageBase, Collection, ContourSet
